In [ ]:
import time
import csv
import pandas as pd
import dask.dataframe as dd



In [ ]:
pip install pandas_schema

In [ ]:
%%time
with open('Parking.csv') as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter=',')
    line_count = 0


In [ ]:
csv_reader

In [ ]:
%%time
data =pd.read_csv("Parking.csv")


In [ ]:
%%time
chunks = pd.read_csv("Parking.csv", chunksize=100000)
data1 = pd.concat(chunks)


In [ ]:
data.head()

In [ ]:
df=data.sample(frac=0.01)


In [ ]:
df

In [ ]:
df.to_csv('test.csv', index=False)

In [ ]:
%%time
df1= dd.read_csv("Parking.csv")



In [ ]:
import os
os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
import ray
import modin.pandas as ray_pd

In [ ]:
%time
df11 = ray_pd.read_csv('Parking.csv')


In [ ]:
data.head()

In [ ]:
data.columns=data.columns.str.replace('[#,@,&]','')
data.head()

In [ ]:
data.columns=data.columns.str.replace(' ','_')
data.head()

In [ ]:
data.columns

In [1]:
%%writefile testutil.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)

def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string


def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting testutil.py


In [2]:
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: test
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    -Summons_Number
    -Plate_ID
    -Registration_State
    -Plate_Type
    -Issue_Date
    -Violation_Code
    -Vehicle_Body_Type
    -Vehicle_Make
    -Issuing_Agency
    -Street_Code1
    -Street_Code2
    -Street_Code3
    -Vehicle_Expiration_Date
    -Violation_Location
    -Violation_Precinct
    -Issuer_Precinct
    -Issuer_Code
    -Issuer_Command
    -Issuer_Squad
    -Violation_Time
    -Time_First_Observed
    -Violation_County
    -Violation_In_Front_Of_Or_Opposite
    -House_Number
    -Street_Name
    -Intersecting_Street
    -Date_First_Observed
    -Law_Section
    -Sub_Division
    -Violation_Legal_Code
    -Days_Parking_In_Effect____
    -From_Hours_In_Effect
    -To_Hours_In_Effect
    -Vehicle_Color
    -Unregistered_Vehicle?
    -Vehicle_Year
    -Meter_Number
    -Feet_From_Curb
    -Violation_Post_Code
    -Violation_Description
    -No_Standing_or_Stopping_Violation
    -Hydrant_Violation
    -Double_Parking_Violation
    -Latitude
    -Longitude
    -Community_Board
    -Community_Council_
    -Census_Tract
    -BIN
    -BBL
    -NTA

Overwriting file.yaml


In [3]:
import testutil as util
config_data = util.read_config_file("file.yaml")

C:\Users\Amshu\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [4]:
config_data['outbound_delimiter']

'|'

In [5]:
config_data

{'file_type': 'csv',
 'dataset_name': 'testfile',
 'file_name': 'test',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': '-Summons_Number -Plate_ID -Registration_State -Plate_Type -Issue_Date -Violation_Code -Vehicle_Body_Type -Vehicle_Make -Issuing_Agency -Street_Code1 -Street_Code2 -Street_Code3 -Vehicle_Expiration_Date -Violation_Location -Violation_Precinct -Issuer_Precinct -Issuer_Code -Issuer_Command -Issuer_Squad -Violation_Time -Time_First_Observed -Violation_County -Violation_In_Front_Of_Or_Opposite -House_Number -Street_Name -Intersecting_Street -Date_First_Observed -Law_Section -Sub_Division -Violation_Legal_Code -Days_Parking_In_Effect____ -From_Hours_In_Effect -To_Hours_In_Effect -Vehicle_Color -Unregistered_Vehicle? -Vehicle_Year -Meter_Number -Feet_From_Curb -Violation_Post_Code -Violation_Description -No_Standing_or_Stopping_Violation -Hydrant_Violation -Double_Parking_Violation -Latitude -Longitude -Co

In [6]:
import pandas as pd
df_sample = pd.read_csv("test.csv",delimiter=',')
df_sample.head()

,Summons_Number,Plate_ID,Registration_State,Plate_Type,Issue_Date,Violation_Code,Vehicle_Body_Type,Vehicle_Make,Issuing_Agency,Street_Code1,...,Hydrant_Violation,Double_Parking_Violation,Latitude,Longitude,Community_Board,Community_Council_,Census_Tract,BIN,BBL,NTA
0,1399710643,U588005,99,999,03/18/2016,66,TRAI,NaN,S,39730,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8225503247,HDP8147,PA,PAS,01/20/2016,38,4DSD,CHRYS,T,20390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4613649845,DPD6702,NY,PAS,12/02/2015,36,SUBN,TOYOT,V,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5089650913,BGM2471,NY,PAS,12/04/2015,7,SUBN,ACURA,V,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4615987110,2929TR,ME,PAS,02/03/2016,36,UT,TOYOT,V,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

C:\Users\Amshu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3325: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,Summons_Number,Plate_ID,Registration_State,Plate_Type,Issue_Date,Violation_Code,Vehicle_Body_Type,Vehicle_Make,Issuing_Agency,Street_Code1,...,Hydrant_Violation,Double_Parking_Violation,Latitude,Longitude,Community_Board,Community_Council_,Census_Tract,BIN,BBL,NTA
0,1399710643,U588005,99,999,03/18/2016,66,TRAI,NaN,S,39730,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8225503247,HDP8147,PA,PAS,01/20/2016,38,4DSD,CHRYS,T,20390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4613649845,DPD6702,NY,PAS,12/02/2015,36,SUBN,TOYOT,V,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5089650913,BGM2471,NY,PAS,12/04/2015,7,SUBN,ACURA,V,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4615987110,2929TR,ME,PAS,02/03/2016,36,UT,TOYOT,V,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['Summons_Number', 'Plate_ID', 'Registration_State', 'Plate_Type',
       'Issue_Date', 'Violation_Code', 'Vehicle_Body_Type', 'Vehicle_Make',
       'Issuing_Agency', 'Street_Code1', 'Street_Code2', 'Street_Code3',
       'Vehicle_Expiration_Date', 'Violation_Location', 'Violation_Precinct',
       'Issuer_Precinct', 'Issuer_Code', 'Issuer_Command', 'Issuer_Squad',
       'Violation_Time', 'Time_First_Observed', 'Violation_County',
       'Violation_In_Front_Of_Or_Opposite', 'House_Number', 'Street_Name',
       'Intersecting_Street', 'Date_First_Observed', 'Law_Section',
       'Sub_Division', 'Violation_Legal_Code', 'Days_Parking_In_Effect____',
       'From_Hours_In_Effect', 'To_Hours_In_Effect', 'Vehicle_Color',
       'Unregistered_Vehicle?', 'Vehicle_Year', 'Meter_Number',
       'Feet_From_Curb', 'Violation_Post_Code', 'Violation_Description',
       'No_Standing_or_Stopping_Violation', 'Hydrant_Violation',
       'Double_Parking_Violation', 'Latitud

In [9]:
util.col_header_val(df,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['street_name', 'issuer_command', 'vehicle_year', 'street_code1', 'to_hours_in_effect', 'issue_date', 'census_tract', 'house_number', 'nta', 'vehicle_body_type', 'violation_location', 'community_board', 'issuer_precinct', 'intersecting_street', 'sub_division', 'violation_precinct', 'bbl', 'street_code2', 'days_parking_in_effect', 'meter_number', 'vehicle_expiration_date', 'feet_from_curb', 'date_first_observed', 'plate_id', 'unregistered_vehicle', 'double_parking_violation', 'law_section', 'vehicle_make', 'violation_county', 'violation_in_front_of_or_opposite', 'from_hours_in_effect', 'violation_time', 'hydrant_violation', 'summons_number', 'street_code3', 'issuer_squad', 'no_standing_or_stopping_violation', 'issuer_code', 'violation_code', 'violation_description', 'violation_post_code', 'registration_state', 'latitude', 'violation_legal_code', 'longitude', 'issuing_agency', 'plate_type', 't

0

In [37]:
df.to_csv('test.csv.gz', compression='gzip', sep='|')


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106269 entries, 0 to 106268
Data columns (total 51 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   summons_number                     106269 non-null  int64  
 1   plate_id                           106248 non-null  object 
 2   registration_state                 106269 non-null  object 
 3   plate_type                         106269 non-null  object 
 4   issue_date                         106269 non-null  object 
 5   violation_code                     106269 non-null  int64  
 6   vehicle_body_type                  105900 non-null  object 
 7   vehicle_make                       105631 non-null  object 
 8   issuing_agency                     106269 non-null  object 
 9   street_code1                       106269 non-null  int64  
 10  street_code2                       106269 non-null  int64  
 11  street_code3                       1062